In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
import pickle
import tensorflow as tf
from sklearn.utils import shuffle
import tsc_utils as tu
import time
import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


SyntaxError: invalid syntax (tsc_utils.py, line 61)

In [3]:
model_list = [
    dict(model = tu.lenet, model_params = tu.params_orig_lenet),
    dict(model = tu.lenet, model_params = tu.params_big_lenet),
    dict(model = tu.sermanet, model_params = tu.params_sermanet),
    dict(model = tu.sermanet, model_params = tu.params_sermanet_big),
    dict(model = tu.sermanet_v2, model_params = tu.params_sermanet_v2)
]
for m in model_list:
    g = tf.Graph()
    with g.as_default():
        model = m['model']
        model_params = m['model_params']

        x = tf.placeholder(tf.float32, (None, 32, 32, 1))
        y = tf.placeholder(tf.int32, (None))
        one_hot_y = tf.one_hot(y, model_params['num_classes'])
        is_training = tf.placeholder(tf.bool)

        logits = model(x, params=model_params, is_training=is_training)
        
    with tf.Session(graph=g) as sess:
        total_parameters = 0
        for variable in tf.trainable_variables():
            # shape is an array of tf.Dimension
            shape = variable.get_shape()
            print(variable)
            print(shape)
            #print(len(shape))
            variable_parametes = 1
            for dim in shape:
                #print(dim)
                variable_parametes *= dim.value
            #print(variable_parametes)
            total_parameters += variable_parametes
        print(total_parameters)
        m['total_parameters']=total_parameters

NameError: name 'tu' is not defined

<tf.Variable 'conv1/weights:0' shape=(5, 5, 1, 64) dtype=float32_ref>
(5, 5, 1, 64)
<tf.Variable 'conv1/biases:0' shape=(64,) dtype=float32_ref>
(64,)
<tf.Variable 'conv2/weights:0' shape=(5, 5, 64, 128) dtype=float32_ref>
(5, 5, 64, 128)
<tf.Variable 'conv2/biases:0' shape=(128,) dtype=float32_ref>
(128,)
<tf.Variable 'conv3/weights:0' shape=(5, 5, 128, 256) dtype=float32_ref>
(5, 5, 128, 256)
<tf.Variable 'conv3/biases:0' shape=(256,) dtype=float32_ref>
(256,)
<tf.Variable 'fc4/weights:0' shape=(7168, 2048) dtype=float32_ref>
(7168, 2048)
<tf.Variable 'fc4/biases:0' shape=(2048,) dtype=float32_ref>
(2048,)
<tf.Variable 'out/weights:0' shape=(2048, 43) dtype=float32_ref>
(2048, 43)
<tf.Variable 'out/biases:0' shape=(43,) dtype=float32_ref>
(43,)
15796267
